# Build a composit model Poisson

A model with Poisson($N_\mathrm{SR} | \mu \times$S + B ) $\times$ Poisson($N_\mathrm{CR} | \tau\times$B)  with $S$=10, tau=10 constant

This will represent a signal region SR and a control region CR (that measures only the bkg)

The constant parameter tau represents the sideband scale factor
 * Generate an observed dataset $N_\mathrm{SR}=25,N_\mathrm{CR}=200$ ;
 * Construct a unique statistical problem definition (RooStats::ModelConfig) that specifies
   * model = model
   * observable = {$N_\mathrm{SR}, N_\mathrm{SR}$}
   * parameter of interest = $\mu$
   * nuisance parameter = $B$
 * Store everything in a root file for later use

In [1]:
%jsroot

Construct an empy workspace that will contain the model

In [2]:
RooWorkspace w("w") ;


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



## Construct a Poisson signal region model

Construct expression for expected event count $N^{\exp}_\mathrm{SR} = \mu \times S + B$, with $S$ fixed and $B$ floating

In [3]:
w.factory("expr::Nexp_SR('mu*S+B',mu[1,-1,10],S[10],B[20,0,200])") ;

Construct Poisson probability model for $N^\mathrm{obs}_\mathrm{SR}$ given $N^\mathrm{exp}_\mathrm{SR}$

In [4]:
w.factory("Poisson::model_SR(Nobs_SR[0,100],Nexp_SR)") ;

## Construct a Poisson control region model 

Construct expression for expected event count $N^\mathrm{exp}_\mathrm{CR} = \tau \times B$, with $\tau$ fixed. The factor $\tau$ expresses the ratio of the CR size w.r.t. the SR size

In [5]:
w.factory("expr::Nexp_CR('tau*B',tau[10],B)") ;

Construct Poisson probability model for $N_\mathrm{obs}$ given $N_\mathrm{exp}$

In [6]:
w.factory("Poisson::model_CR(Nobs_CR[0,1000],Nexp_CR)") ;

## Construct a joint model for the SR and CR

In [7]:
w.factory("PROD::model(model_SR,model_CR)") ;

## Construct a dataset containing a single observed event count

For convenience define a named set in the workspace obs = ($N^\mathrm{obs}_\mathrm{SR},N^\mathrm{obs}_\mathrm{CR}$)

In [8]:
w.defineSet("obs","Nobs_SR,Nobs_CR") ;

Construct an empty dataset for observable Nobs

In [9]:
RooDataSet d("d","d",*w.set("obs")) ;

Set Nobs to desired observed event count

In [10]:
w.var("Nobs_SR")->setVal(25) ;
w.var("Nobs_CR")->setVal(200) ;

Add observation of 25 events to dataset

In [11]:
d.add(*w.set("obs")) ;
w.import(d,RooFit::Rename("observed_data")) ;

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset d
[#1] INFO:ObjectHandling -- RooWorkSpace::import(w) changing name of dataset from  d to observed_data


## Construct a unique statistical interpretatation of the model to be used by RooStats

Create an empty ModelConfig

In [12]:
RooStats::ModelConfig mc("ModelConfig",&w);

Define the pdf, the parameter of interest and the observables

In [13]:
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("mu"));
mc.SetNuisanceParameters(*w.var("B"));
mc.SetObservables(*w.set("obs"));

Define the current value $\mu=1$ as an hypothesis 

In [14]:
mc.SetSnapshot(*w.var("mu"));
mc.Print();


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR,Nobs_CR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (B)
PDF:                     RooProdPdf::model[ model_SR * model_CR ] = 0.00144134
Snapshot:                
  1) 0x7f01809f6cb0 RooRealVar:: mu = 1  L(-1 - 10)  "mu"



import model in the workspace

In [15]:
w.import(mc) ;
w.writeToFile("../workspaces/model.root") ;
w.pdf("model")->fitTo(d) ;

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_Nobs_CR:Nobs_SR with 0 entries
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (model_SR,model_CR)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 B            2.00000e+01  1.00000e+01    0.00000e+00  2.00000e+02
     2 mu           1.00000e+00  1.00000e+00   -1.00000e+00  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT,